<a href="https://colab.research.google.com/github/Nevesm5/Datario/blob/main/analise_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
pip install basedosdados

In [18]:
import basedosdados as bd
import pandas as pd
import numpy as np

In [19]:
bairro = bd.read_sql("SELECT * FROM datario.dados_mestres.bairro", billing_project_id= "746089026690", reauth=True)



Downloading: 100%|██████████| 164/164 [00:01<00:00, 106.74rows/s]


Tabelas (1 a 5) \
datario.dados_mestres.bairro \
datario.adm_central_atendimento_1746.chamado

1-Quantos chamados foram abertos no dia 01/04/2023?

In [20]:
chamado = bd.read_sql("SELECT * FROM datario.adm_central_atendimento_1746.chamado where date(data_inicio)='2023-04-01'", billing_project_id= "746089026690")


Downloading: 100%|██████████| 1756/1756 [00:01<00:00, 1616.35rows/s]


In [21]:
chamado.shape[0]

1756

2-Qual o tipo de chamado que teve mais teve chamados abertos no dia 01/04/2023?

In [22]:
chamado.tipo.value_counts().reset_index().rename(columns={'index':'tipo', 'tipo':'quantidade'}).head(1)

,tipo,quantidade
0,Estacionamento irregular,366


3-Quais os nomes dos 3 bairros que mais tiveram chamados abertos nesse dia?

In [23]:
np.sort(chamado.columns)

array(['categoria', 'data_alvo_diagnostico', 'data_alvo_finalizacao',
       'data_fim', 'data_inicio', 'data_particao',
       'data_real_diagnostico', 'dentro_prazo', 'id_bairro', 'id_chamado',
       'id_logradouro', 'id_origem_ocorrencia', 'id_subtipo',
       'id_territorialidade', 'id_tipo', 'id_unidade_organizacional',
       'id_unidade_organizacional_mae', 'justificativa_status',
       'latitude', 'longitude', 'nome_unidade_organizacional',
       'numero_logradouro', 'prazo_tipo', 'prazo_unidade', 'reclamacoes',
       'situacao', 'status', 'subtipo', 'tempo_prazo', 'tipo',
       'tipo_situacao', 'unidade_organizacional_ouvidoria'], dtype=object)

In [24]:
bairro.columns

Index(['id_bairro', 'nome', 'id_area_planejamento', 'id_regiao_planejamento',
       'nome_regiao_planejamento', 'id_regiao_administrativa',
       'nome_regiao_administrativa', 'subprefeitura', 'area', 'perimetro',
       'geometry_wkt', 'geometry'],
      dtype='object')

In [25]:
chamado.merge(bairro, on='id_bairro')['nome'].value_counts().head(3)

,nome
Campo Grande,113
Tijuca,89
Barra da Tijuca,59


In [26]:
chamado.merge(bairro, on='id_bairro').groupby('nome').size().sort_values(ascending=False).head(3)

,0
nome,
Campo Grande,113
Tijuca,89
Barra da Tijuca,59


4 - Qual o nome da subprefeitura com mais chamados abertos nesse dia?

In [27]:
bairro.columns

Index(['id_bairro', 'nome', 'id_area_planejamento', 'id_regiao_planejamento',
       'nome_regiao_planejamento', 'id_regiao_administrativa',
       'nome_regiao_administrativa', 'subprefeitura', 'area', 'perimetro',
       'geometry_wkt', 'geometry'],
      dtype='object')

In [28]:
chamado.merge(bairro, on='id_bairro')['subprefeitura'].value_counts().head(1)

,subprefeitura
Zona Norte,510


5 - Existe algum chamado aberto nesse dia que não foi associado a um bairro ou subprefeitura na tabela de bairros? Se sim, por que isso acontece?

In [29]:
chamado_bairro = chamado.merge(bairro, on='id_bairro')

In [30]:
chamado_bairro.columns

Index(['id_chamado', 'id_origem_ocorrencia', 'data_inicio', 'data_fim',
       'id_bairro', 'id_territorialidade', 'id_logradouro',
       'numero_logradouro', 'id_unidade_organizacional',
       'nome_unidade_organizacional', 'id_unidade_organizacional_mae',
       'unidade_organizacional_ouvidoria', 'categoria', 'id_tipo', 'tipo',
       'id_subtipo', 'subtipo', 'status', 'longitude', 'latitude',
       'data_alvo_finalizacao', 'data_alvo_diagnostico',
       'data_real_diagnostico', 'tempo_prazo', 'prazo_unidade', 'prazo_tipo',
       'dentro_prazo', 'situacao', 'tipo_situacao', 'justificativa_status',
       'reclamacoes', 'data_particao', 'nome', 'id_area_planejamento',
       'id_regiao_planejamento', 'nome_regiao_planejamento',
       'id_regiao_administrativa', 'nome_regiao_administrativa',
       'subprefeitura', 'area', 'perimetro', 'geometry_wkt', 'geometry'],
      dtype='object')

In [31]:
for i in chamado_bairro.columns:
  if  (chamado_bairro[i].isna().sum()) > 1:
    print (i)

data_fim
numero_logradouro
longitude
latitude
data_alvo_finalizacao
data_alvo_diagnostico
data_real_diagnostico
tempo_prazo
justificativa_status


Nova base chamados e eventos

In [32]:
chamado = bd.read_sql("SELECT * FROM datario.adm_central_atendimento_1746.chamado \
where date(data_inicio) between '2022-01-01' and '2023-12-31'  \
", billing_project_id= "746089026690")


Downloading: 100%|██████████| 1634336/1634336 [09:04<00:00, 3002.16rows/s]


In [33]:
eventos = bd.read_sql("SELECT * FROM datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos", billing_project_id= "746089026690")


Downloading: 100%|██████████| 8/8 [00:00<00:00, 39.30rows/s]


In [34]:
chamado.head(2)

,id_chamado,id_origem_ocorrencia,data_inicio,data_fim,id_bairro,id_territorialidade,id_logradouro,numero_logradouro,id_unidade_organizacional,nome_unidade_organizacional,...,data_real_diagnostico,tempo_prazo,prazo_unidade,prazo_tipo,dentro_prazo,situacao,tipo_situacao,justificativa_status,reclamacoes,data_particao
0,17556781,11,2022-08-04 00:58:17,2022-09-22 17:47:21,None,None,None,<NA>,8,RIOLUZ - Companhia Municipal de Energia e Ilum...,...,NaT,<NA>,D,F,Fora do prazo,Encerrado,Não atendido,None,0,2022-08-01
1,17561271,1,2022-08-04 21:32:27,2022-08-05 23:43:00,6,1,157610,152,8,RIOLUZ - Companhia Municipal de Energia e Ilum...,...,NaT,<NA>,D,F,No prazo,Encerrado,Atendido,None,0,2022-08-01


In [35]:
eventos

,ano,data_inicial,data_final,evento,taxa_ocupacao
0,02/09 a 04/09 de 2022,2022-09-02,2022-09-04,Rock in Rio,0.8184
1,02/09 a 04/09 de 2022,2022-09-02,2022-09-04,Rock in Rio,0.8184
2,08/09 a 11/09 de 2022,2022-09-08,2022-09-11,Rock in Rio,0.9451
3,08/09 a 11/09 de 2022,2022-09-08,2022-09-11,Rock in Rio,0.9451
4,18/02 a 21/02 de 2023,2023-02-18,2023-02-21,Carnaval,0.9554
5,18/02 a 21/02 de 2023,2023-02-18,2023-02-21,Carnaval,0.9554
6,30-31/12 e 01/01 (2022-2023),2022-12-30,2023-01-01,Reveillon,0.9251
7,30-31/12 e 01/01 (2022-2023),2022-12-30,2023-01-01,Reveillon,0.9251


In [36]:
query = """SELECT
  *
FROM
  datario.adm_central_atendimento_1746.chamado AS A
JOIN
  datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos AS B
ON
   A.data_inicio = B.data_inicial """

In [37]:
bd.read_sql(query,  billing_project_id= "746089026690")







Downloading: 0rows [00:00, ?rows/s]
Downloading: 0rows [00:00, ?rows/s]


,id_chamado,id_origem_ocorrencia,data_inicio,data_fim,id_bairro,id_territorialidade,id_logradouro,numero_logradouro,id_unidade_organizacional,nome_unidade_organizacional,...,situacao,tipo_situacao,justificativa_status,reclamacoes,data_particao,ano,data_inicial,data_final,evento,taxa_ocupacao


6-Quantos chamados com o subtipo "Perturbação do sossego" foram abertos desde 01/01/2022 até 31/12/2023 (incluindo extremidades)?

In [38]:
chamado[chamado['subtipo']=='Perturbação do sossego'].shape[0]

42830

7-Selecione os chamados com esse subtipo que foram abertos durante os eventos contidos na tabela de eventos (Reveillon, Carnaval e Rock in Rio).

In [39]:
pertubacao = chamado[chamado['subtipo']=='Perturbação do sossego']

In [40]:
eventos.data_inicial = eventos.data_inicial.astype(str)

In [41]:
pertubacao['data_inicial'] =  pertubacao['data_inicio'].astype(str).apply(lambda x : x[0:10])

In [42]:
pert_evento = pertubacao.merge(eventos, on='data_inicial')

In [43]:
pert_evento['evento'].unique()


array(['Rock in Rio', 'Carnaval', 'Reveillon'], dtype=object)

In [44]:
pert_evento['id_chamado']


,id_chamado
0,17659216
1,17659216
2,17661662
3,17661662
4,17661644
...,...
585,18078353
586,18077762
587,18077762
588,18075910


8-Quantos chamados desse subtipo foram abertos em cada evento?

In [45]:
pert_evento['evento'].value_counts()

,evento
Rock in Rio,354
Carnaval,148
Reveillon,88


9-Qual evento teve a maior média diária de chamados abertos desse subtipo?

In [46]:
pert_evento.groupby(['evento','data_inicial']).sum()

numero_logradouro    longitude     latitude  \
evento      data_inicial                                                
Carnaval    2023-02-18               259598 -5282.651037 -2798.196840   
Reveillon   2022-12-30                24902 -3115.311210 -1650.330504   
Rock in Rio 2022-09-02               154588 -7445.698706 -3943.729307   
            2022-09-08                53262 -5628.396087 -2980.421585   

                          tempo_prazo  reclamacoes  taxa_ocupacao  
evento      data_inicial                                           
Carnaval    2023-02-18              0            8       141.3992  
Reveillon   2022-12-30              0           10        81.4088  
Rock in Rio 2022-09-02              0           30       163.6800  
            2022-09-08              0           18       145.5454

In [47]:
pert_evento[['id_chamado', 'evento', 'data_inicial']].groupby(['evento', 'data_inicial']).count().groupby('evento').mean().sort_values(by='id_chamado', ascending=False)

,id_chamado
evento,
Rock in Rio,177.0
Carnaval,148.0
Reveillon,88.0


10-Compare as médias diárias de chamados abertos desse subtipo durante os eventos específicos (Reveillon, Carnaval e Rock in Rio) e a média diária de chamados abertos desse subtipo considerando todo o período de 01/01/2022 até 31/12/2023.

In [48]:

media_diaria_eventos = pert_evento[['id_chamado', 'evento', 'data_inicial']].groupby(['evento', 'data_inicial']).count().groupby('evento').mean().sort_values(by='id_chamado', ascending=False)
media_diaria_total = pertubacao.shape[0] / ((pd.to_datetime('2023-12-31') - pd.to_datetime('2022-01-01')).days + 1)
print("Média diária de chamados 'Perturbação do sossego' durante os eventos:")
print(media_diaria_eventos)
print("\nMédia diária de chamados 'Perturbação do sossego' no período total:")
"imprimindo a media_diaria_total com 2 casas decimais"

print(f"{media_diaria_total:.2f}")


Média diária de chamados 'Perturbação do sossego' durante os eventos:
             id_chamado
evento                 
Rock in Rio       177.0
Carnaval          148.0
Reveillon          88.0

Média diária de chamados 'Perturbação do sossego' no período total:
58.67
